### Week 7
### DSC 650
### Abed Tabbalat

In [1]:
# Mounting drive
import os
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.chdir('/content/drive/My Drive/DSC650/Original/dsc650/assignments/assignment07')
!pwd

Mounted at /content/drive
/content/drive/My Drive/DSC650/Original/dsc650/assignments/assignment07


In [4]:
!pip install pygeohash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6164 sha256=26efca7ae575c303b1b546563fcc7959b9db87c7b90e9411b769c1c922c436ac
  Stored in directory: /root/.cache/pip/wheels/6e/e7/a2/4ea56930e1cc3ece852306b842f9ca71e0cd9482cb8b848e27
Successfully built pygeohash


In [28]:
# Imports
import os
import json
from pathlib import Path
import gzip
import shutil
import pandas as pd
import pygeohash
import hashlib
import itertools

In [10]:
# Set the directories for the results and the source data
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')

# Delete the results directory if it already exists, and then create it
if results_dir.exists():
    shutil.rmtree(results_dir)

results_dir.mkdir(parents=True, exist_ok=True)

# Set the path to the source data
src_data_path = '/content/drive/My Drive/DSC650/Original/data/processed/openflights/routes.jsonl.gz'

## Functions

In [8]:
# Define a function to read the source data from a jsonl.gz file
def read_jsonl_data():
    with gzip.open(src_data_path, 'rb') as f:
        # Read the lines from the file and parse them as JSON objects
        records = [json.loads(line) for line in f.readlines()]
        
    return records

# Define a function to flatten a record
def flatten_record(record):
    flat_record = dict()
    
    # Loop through the keys and values in the record
    for key, value in record.items():
        # If the value is a dictionary, loop through its keys and values
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            # Otherwise, add the key and value to the flattened record
            flat_record[key] = value
    
    return flat_record

# Define a function to create a flattened dataset
def create_flattened_dataset():
    # Read the source data
    records = read_jsonl_data()
    
    # Set the path to the output Parquet file
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    
    # Create a pandas DataFrame from the flattened records
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

## Importing Data

In [12]:
df = create_flattened_dataset()
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_latitude,dst_airport_longitude,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,44.225101,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,55.012600,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,-34.945000,138.531006,20.0,9.5,O,Australia/Adelaide,airport,OurAirports,False,[SF3]
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,43.061298,74.477600,2058.0,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734]
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,55.408798,37.906300,588.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[734]
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,40.609001,72.793297,2927.0,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734]


## Assignment 7.1

### Part A

In [15]:
# Add a new column to the DataFrame that combines three other columns into a single key
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)
df['key']

0        AERKZN2B
1        ASFKZN2B
2        ASFMRV2B
3        CEKKZN2B
4        CEKOVB2B
           ...   
67658    WYAADLZL
67659    DMEFRUZM
67660    FRUDMEZM
67661    FRUOSSZM
67662    OSSFRUZM
Name: key, Length: 67663, dtype: object

In [16]:
# Define a list of tuples that represent the partitions
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

# Set up a dictionary that maps each partition to a key
partition_dict = {}

for i in partitions:
    if i[0] == i[1]:
        partition_dict[i] = i[0]
    else:
        partition_dict[i] = i[0] + '-' + i[1]
        
partition_dict

{('A', 'A'): 'A',
 ('B', 'B'): 'B',
 ('C', 'D'): 'C-D',
 ('E', 'F'): 'E-F',
 ('G', 'H'): 'G-H',
 ('I', 'J'): 'I-J',
 ('K', 'L'): 'K-L',
 ('M', 'M'): 'M',
 ('N', 'N'): 'N',
 ('O', 'P'): 'O-P',
 ('Q', 'R'): 'Q-R',
 ('S', 'T'): 'S-T',
 ('U', 'U'): 'U',
 ('V', 'V'): 'V',
 ('W', 'X'): 'W-X',
 ('Y', 'Z'): 'Y-Z'}

In [18]:
# Define a function that generates a kv_key from a data_key
def kv_key_gen(data_key):
    for key, val in partition_dict.items():
        if data_key[0] == key[0] or data_key[0] == key[1]:
            return val
        
    return ''

# Add a new column to the DataFrame that contains the kv_key values
df['kv_key'] = df['key'].apply(kv_key_gen)
df[['kv_key', 'key']]

,kv_key,key
0,A,AERKZN2B
1,A,ASFKZN2B
2,A,ASFMRV2B
3,C-D,CEKKZN2B
4,C-D,CEKOVB2B
...,...,...
67658,W-X,WYAADLZL
67659,C-D,DMEFRUZM
67660,E-F,FRUDMEZM
67661,E-F,FRUOSSZM


In [19]:
df.to_parquet(results_dir.joinpath('kv'), partition_cols = ['kv_key'])

### Part B

In [21]:
# Define a function that generates a hashed value from a key
def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    
    return m.hexdigest()

# Add two new columns to the DataFrame that contain the hashed value and the first character of the hash
df['hashed'] = df['key'].apply(hash_key)
df['hash_key'] = df['hashed'].str[0]
df[['key', 'hash_key', 'hashed']]

,key,hash_key,hashed
0,AERKZN2B,6,652cdec02010381f175efe499e070c8cbaac1522bac59a...
1,ASFKZN2B,9,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...
2,ASFMRV2B,1,161143856af25bd4475f62c80c19f68936a139f653c1d3...
3,CEKKZN2B,3,39aa99e6ae2757341bede9584473906ef1089e30820c90...
4,CEKOVB2B,1,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...
...,...,...,...
67658,WYAADLZL,f,f31527be84c36208c05cac57dfac8a46b48a87dda151f8...
67659,DMEFRUZM,8,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...
67660,FRUDMEZM,e,e976939986fbf947bb9318018cef717c0b34dff91e5e67...
67661,FRUOSSZM,8,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...


In [22]:
df.to_parquet(results_dir.joinpath('hash'), partition_cols=['hash_key'])

### Part C

In [24]:
# Add a new column to the DataFrame that contains the geohash values for each airport
df['airport_geohash'] = df.apply(lambda x: pygeohash.encode(x['src_airport_latitude'], 
                                                            x['src_airport_longitude']), 
                                 axis = 1)
df[['src_airport_latitude', 'src_airport_longitude', 'airport_geohash']]

,src_airport_latitude,src_airport_longitude,airport_geohash
0,43.449902,39.956600,szsrjjzd02b3
1,46.283298,48.006302,v04pk3t5gbjj
2,46.283298,48.006302,v04pk3t5gbjj
3,55.305801,61.503300,v3gdxs17du83
4,55.305801,61.503300,v3gdxs17du83
...,...,...,...
67658,-33.058899,137.514008,r41gcjy9uwef
67659,55.408798,37.906300,ucfgnwfe8u9e
67660,43.061298,74.477600,txsuyz0fjzgd
67661,43.061298,74.477600,txsuyz0fjzgd


In [25]:
# Define a dictionary that contains the geohash values for the data centers
data_centers = dict(
    west = pygeohash.encode(45.5945645, -121.1786823),
    central = pygeohash.encode(41.1544433, -96.0422378),
    east = pygeohash.encode(39.08344, -77.6497145)
)

print(data_centers)


{'west': 'c21g6s0rs4c7', 'central': '9z7dnebnj8kb', 'east': 'dqby34cjw922'}


In [26]:
# Define a function called `get_datacenter_location()` that finds the closest data center to a given geohash
def get_datacenter_location(geohash): 
    distance_dict = {}
    
    # Iterate through each data center and calculate the distance to the given geohash
    for key in data_centers.keys():
        distance_dict[key] = pygeohash.geohash_haversine_distance(data_centers.get(key), geohash)
        
    # Sort the distance dictionary by value and return the key with the lowest distance
    sorted_distance = sorted(distance_dict.items(), key = lambda x: x[1])[0][0]
        
    return sorted_distance

# Apply the `get_datacenter_location()` function to the `airport_geohash` column of the DataFrame to find the closest data center to each source airport
df['data_center'] = df['airport_geohash'].apply(lambda x: get_datacenter_location(x))

df[['src_airport_latitude', 'src_airport_longitude', 'airport_geohash', 'data_center']]


,src_airport_latitude,src_airport_longitude,airport_geohash,data_center
0,43.449902,39.956600,szsrjjzd02b3,east
1,46.283298,48.006302,v04pk3t5gbjj,east
2,46.283298,48.006302,v04pk3t5gbjj,east
3,55.305801,61.503300,v3gdxs17du83,west
4,55.305801,61.503300,v3gdxs17du83,west
...,...,...,...,...
67658,-33.058899,137.514008,r41gcjy9uwef,west
67659,55.408798,37.906300,ucfgnwfe8u9e,east
67660,43.061298,74.477600,txsuyz0fjzgd,west
67661,43.061298,74.477600,txsuyz0fjzgd,west


In [27]:
df.to_parquet(results_dir.joinpath('geo'), partition_cols=['data_center'])

### Part D

In [29]:
# Define a function called `balance_partitions()` that balances the number of keys in each partition
def balance_partitions(keys, num_partitions):
    # Calculate the partition size and create an iterator for the keys
    partition_size = round(len(keys) / num_partitions)
    iterations = iter(keys)
    
    # Create an iterator that returns tuples of keys up to the partition size
    partitions_iterations = iter(lambda: tuple(itertools.islice(iterations, partition_size)), ())
    
    # Sort each partition and return the list of partitions
    partitions = [sorted(part) for part in partitions_iterations]
    
    return partitions

# Test the `balance_partitions()` function by creating five partitions with 50 airline names
airline_names = df.airline_iata.sample(50).to_list()
partitions = balance_partitions(airline_names, 5)

partitions


[['AA', 'CZ', 'DL', 'FD', 'HX', 'NH', 'NZ', 'OS', 'US', 'ZI'],
 ['4U', 'AB', 'DL', 'GA', 'HU', 'IG', 'KN', 'SQ', 'V7', 'WS'],
 ['CA', 'GS', 'IR', 'JL', 'MI', 'Q6', 'RJ', 'SA', 'SV', 'XY'],
 ['5N', 'AK', 'AS', 'AV', 'BT', 'CA', 'HG', 'NZ', 'VA', 'WN'],
 ['BA', 'G4', 'LH', 'MH', 'MO', 'RJ', 'TK', 'UA', 'UA', 'US']]